In [12]:
import pandas as pd
import kagglehub
import re


from kagglehub import KaggleDatasetAdapter

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.model_selection import train_test_split
from datasets import load_dataset

from sklearn.metrics import classification_report

In [2]:
file_path = "Twitter_Data.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "saurabhshahane/twitter-sentiment-dataset",
  file_path,
)

print("First 5 records:", df.head())

/tmp/ipykernel_80051/2742394549.py:4: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:                                           clean_text  category
0  when modi promised “minimum government maximum...      -1.0
1  talk all the nonsense and continue all the dra...       0.0
2  what did just say vote for modi  welcome bjp t...       1.0
3  asking his supporters prefix chowkidar their n...       1.0
4  answer who among these the most powerful world...       1.0


In [3]:
# negative(-1), neutral(0), and positive(+1)
df.head(5)

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [5]:
df['clean_text'][2]

'what did just say vote for modi  welcome bjp told you rahul the main campaigner for modi think modi should just relax'

In [7]:
def text_preprocessing(sentiment_df):
    # Create a new column for the processed text
    sentiment_df['processed_text'] = ''

    for index, row in sentiment_df.iterrows():
        text = row['clean_text']
        if not isinstance(text, str):
            # If not a string, replace with empty string or default value
            print(text)
            text = ''
        else:
            text = text.lower()
        
        # Remove links
        text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', '', text)
        # Remove usernames
        text = re.sub(r'@[^\s]+', '', text)
        # Replace hashtags with just the word
        text = re.sub(r'#([^\s]+)', r'\1', text)
        # Fix multiple white spaces to a single space
        text = re.sub(r'[\s]+', ' ', text)
        # Remove words shorter than 4 characters and strip spaces
        text = re.sub(r'\W*\b\w{1,3}\b', '', text)
        text = text.strip()

        # Save the cleaned text in the DataFrame
        sentiment_df.at[index, 'processed_text'] = text


In [8]:
# Df cleaning
text_preprocessing(df)

nan
nan
nan
nan


In [10]:
df.head(10)

,clean_text,category,processed_text
0,when modi promised “minimum government maximum...,-1.0,when modi promised “minimum government maximum...
1,talk all the nonsense and continue all the dra...,0.0,talk nonsense continue drama will vote modi
2,what did just say vote for modi welcome bjp t...,1.0,what just vote modi welcome told rahul main ca...
3,asking his supporters prefix chowkidar their n...,1.0,asking supporters prefix chowkidar their names...
4,answer who among these the most powerful world...,1.0,answer among these most powerful world leader ...
5,kiya tho refresh maarkefir comment karo,0.0,kiya refresh maarkefir comment karo
6,surat women perform yagna seeks divine grace f...,0.0,surat women perform yagna seeks divine grace n...
7,this comes from cabinet which has scholars lik...,0.0,this comes from cabinet which scholars like mo...
8,with upcoming election india saga going import...,1.0,with upcoming election india saga going import...
9,gandhi was gay does modi,1.0,gandhi does modi


In [ ]:
X = df['processed_text']
y = df['category']

In [ ]:
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=42)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=42) 

print(x_train, x_val, x_test)

In [ ]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

  2025-10-16T09:59:03.321806Z ERROR  Python exception updating progress:, error: PyErr { type: <class 'LookupError'>, value: LookupError(<ContextVar name='shell_parent' at 0x71614c03abb0>), traceback: Some(<traceback object at 0x715fe6a3fc80>) }, caller: "src/progress_update.rs:313"
    at /home/runner/work/xet-core/xet-core/error_printer/src/lib.rs:28

